# 1. Основы PySpark

**Всегда закрывайте сессию после работы**
```
spark.stop()
```

Установите Java (обязательно!). Скачайте Amazon Corretto 11 (или OpenJDK 11).  
```
# Проверка (в PowerShell):
java -version
```
Установите Python 3.11. Скачайте с официального сайта. При установке отметьте "Add Python to PATH".  
```
python --version
pip --version
```
Установите Hadoop и Spark. Скачайте Spark 3.5.x (предпочтительно) и Hadoop 3.3. Распакуйте архив в C:\spark (или другой путь без пробелов). Скачайте winutils.exe для Hadoop в папку C:\hadoop\bin. Добавьте переменные среды:
```
[System.Environment]::SetEnvironmentVariable("HADOOP_HOME", "C:\hadoop", "Machine")
[System.Environment]::SetEnvironmentVariable("SPARK_HOME", "C:\spark", "Machine")
[System.Environment]::SetEnvironmentVariable("PATH", "$env:PATH;C:\spark\bin;C:\hadoop\bin", "Machine")
```

Установите PySpark и зависимости
```
pip install pyspark pandas jupyter pyarrow
```

Проверка установки. Тест PySpark в Jupyter
```
jupyter notebook
```
В ноутбуке выполните
```
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Test") \
    .getOrCreate()
df = spark.createDataFrame([(1, "Alice"), (2, "Bob")], ["id", "name"])
df.show()
```
Вывод

| id| name|
| -- | -- |
|  1|Alice|
|  2|  Bob|

**Используемые датасеты**  
[Titanic](https://www.kaggle.com/competitions/titanic/data?select=train.csv "CSV")  
[Amazon Sales Data](https://www.kaggle.com/datasets/karkavelrajaj/amazon-sales-dataset "JSON")  
[COVID-19 Dataset](https://www.kaggle.com/datasets/imdevskp/corona-virus-report "Parquet")  

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Test") \
    .getOrCreate()
df = spark.createDataFrame([(1, "Alice"), (2, "Bob")], ["id", "name"])
df.show()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
+---+-----+



In [5]:
import os
os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['HADOOP_USER_NAME'] = 'root'  # Обход проверки пользователя

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Test") \
    .master("local[*]") \
    .config("spark.driver.host", "localhost") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

try:
    df = spark.createDataFrame([(1, "Alice"), (2, "Bob")], ["id", "name"])
    df.show()
finally:
    spark.stop()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
+---+-----+



## 1.1. Введение в Spark и RDD

PySpark — это Python API для Apache Spark — высокопроизводительной распределённой платформы для обработки больших данных. Позволяет писать параллельные программы, которые работают на кластерах, но при этом использовать знакомый Python. Spark работает с большими объёмами данных, поддерживает SQL, машинное обучение, потоковую обработку. Jupyter Notebook — удобный инструмент для интерактивного написания и запуска кода, идеально подходит для обучения и прототипирования.

RDD(Resilient Distributed Dataset) - базовая структура данных в Spark. **Неизменяемая:** После создания нельзя изменить, только преобразовать. **Распределённая:** Данные разделены на части (партиции) и обрабатываются параллельно.  
SparkSession – точка входа для работы с Spark (аналог SparkContext в старых версиях).

Ключевые характеристики RDD:
- Resilient (Устойчивость). RDD автоматически восстанавливается при сбоях благодаря lineage (цепочке преобразований). Если часть данных теряется, Spark пересчитывает её заново на основе исходных данных и применённых операций.
- Distributed (Распределённость). Данные разбиваются на партиции (partitions) и распределяются по узлам кластера. Обработка происходит параллельно.
- Dataset (Набор данных). Может содержать данные любого типа (числа, строки, объекты и т. д.). Поддерживает операции: map, filter, reduce, join и др.

Как создаются RDD?  
```
# Из внешних данных (HDFS, S3, локальная файловая система):
rdd = sc.textFile("hdfs://path/to/file.txt")

# Из коллекций в памяти (например, список Python):
rdd = sc.parallelize([1, 2, 3, 4, 5])

# Преобразованием другого RDD (например, map, filter):
rdd2 = rdd.map(lambda x: x * 2)
```

### Пример:

Создание RDD из списка

In [3]:
from pyspark.sql import SparkSession

# Инициализация SparkSession
spark = SparkSession.builder \
    .appName("RDD Basics") \
    .master("local[*]") \
    .getOrCreate()

# Создание RDD из списка чисел
data = [1, 2, 3, 4, 5]
rdd = spark.sparkContext.parallelize(data)

# Трансформация: умножение каждого элемента на 2
rdd_doubled = rdd.map(lambda x: x * 2)

# Действие: сбор результатов
print(rdd_doubled.collect())  # Вывод: [2, 4, 6, 8, 10]

[2, 4, 6, 8, 10]


Фильтрация данных

In [6]:
# Фильтрация чётных чисел
rdd_even = rdd.filter(lambda x: x % 2 == 0)
print(rdd_even.collect())  # Вывод: [2, 4]

[2, 4]


Работа с текстом

In [8]:
text_data = ["Hello Spark", "Hello Python", "Hello World"]
text_rdd = spark.sparkContext.parallelize(text_data)

# Разбиваем строки на слова
words_rdd = text_rdd.flatMap(lambda line: line.split(" "))

# Подсчёт слов
word_counts = words_rdd.countByValue()
print(word_counts)  # Вывод: {'Hello': 3, 'Spark': 1, 'Python': 1, 'World': 1}

defaultdict(<class 'int'>, {'Hello': 3, 'Spark': 1, 'Python': 1, 'World': 1})


## 1.2. DataFrames (основы)

PySpark DataFrame - это распределенная коллекция данных, организованная в именованные столбцы, предоставляемая Apache Spark для работы с большими данными в Python. Это аналог pandas DataFrame, но предназначенный для работы в распределенной среде.

**Основные характеристики**. *Распределенная структура*: Данные разделены между узлами кластера. *Неизменяемость*: DataFrame иммутабельны, все операции создают новые DataFrame. *Ленивые вычисления*: Операции выполняются только при вызове действия (action). *Оптимизация*: Использует Catalyst Optimizer для оптимизации запросов.

Отличие от RDD:
- Оптимизирован для SQL-подобных операций.
- Имеет схему (типы данных колонок).

Основные методы:
- show() – вывод данных.
- printSchema() – вывод структуры.
- select(), filter(), groupBy(), agg().

### Практика

Воспользуемся датасет Titanic (файл train.csv)

In [6]:
spark = SparkSession.builder \
    .appName("Titanic Analysis") \
    .master("local[*]") \
    .getOrCreate()

# Загрузка CSV с заголовком и выводом схемы
df = spark.read.csv("data/train.csv", header=True, inferSchema=True)
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



Исследование данных

In [13]:
# Показать первые 5 строк
df.show(5)

# Статистика по числовым колонкам
df.describe(["Age", "Fare"]).show()

# Количество выживших и погибших
df.groupBy("Survived").count().show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

Фильтрация и агрегация. 

In [10]:
from pyspark.sql.functions import avg, max  # Импортируем нужные функции

# Стоимость билета по классам
df.groupBy("Pclass").agg(avg("Fare").alias("AvgFare"), max("Fare").alias("MaxFare")).show()

# Средний возраст выживших женщин.
df.filter((df.Sex == "female") & (df.Survived == 1)).select(avg("Age")).show()

+------+------------------+--------+
|Pclass|           AvgFare| MaxFare|
+------+------------------+--------+
|     1| 84.15468749999992|512.3292|
|     3|13.675550101832997|   69.55|
|     2| 20.66218315217391|    73.5|
+------+------------------+--------+

+-----------------+
|         avg(Age)|
+-----------------+
|28.84771573604061|
+-----------------+



In [28]:
from pyspark.sql.functions import min  # Импортируем нужные функции

# количество пассажиров в каждом классе (Pclass).
df.groupBy("Pclass").count().show()

# минимальный и максимальный возраст пассажиров
df.agg(min("Age").alias("min_age"), max("Age").alias("min_age")).show()

+------+-----+
|Pclass|count|
+------+-----+
|     1|  216|
|     3|  491|
|     2|  184|
+------+-----+

+-------+-------+
|min_age|min_age|
+-------+-------+
|   0.42|   80.0|
+-------+-------+



###  JSON в PySpark

JSON – полуструктурированный формат, поддерживающий вложенные поля. Spark автоматически определяет схему, но для сложных структур её лучше задавать вручную. Вложенные поля обрабатываются через . (например, `user.address.city`).

Функции для работы с JSON:
- from_json() - Преобразует JSON-строку в структуру.
- to_json() - Преобразует структуру в JSON-строку.
- json_tuple() - Извлекает элементы из JSON-строки.
- get_json_object() - Извлекает элемент по JSON-пути.

## 1.3. Базовые операции с DataFrames

## 1.4. Работа с дубликатами и пропусками

## 1.5. SQL-синтаксис в PySpark

In [11]:
spark.stop()